<a href="https://colab.research.google.com/github/tuanng007/medical_qa_gpt2/blob/main/Medical_QA_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%
# @title 1. Install dependencies and setting Kaggle API
!pip install -q transformers datasets torch accelerate kaggle gradio sentencepiece pandas fuzzywuzzy[speedup] evaluate rouge-score

import os
import json
from zipfile import ZipFile
import pandas as pd
from google.colab import files
import random
import torch
import evaluate
import pandas as pd
import re
from datasets import load_dataset, Dataset, DatasetDict

# Upload kaggle.json
print("Please upload your's file 'kaggle.json' .")
if not os.path.exists('/root/.kaggle/kaggle.json'):
    uploaded = files.upload()
    if 'kaggle.json' in uploaded:
        print("Đã upload 'kaggle.json'. Đang cấu hình...")
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"
        print("Kaggle API key configured.")
    else:
        print("Lỗi: Không tìm thấy file 'kaggle.json' đã upload.")
else:
    print("'kaggle.json' đã tồn tại. Bỏ qua bước upload.")
    os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# %%

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.8 MB/s eta 0:00:00
   

Saving kaggle.json to kaggle.json
Đã upload 'kaggle.json'. Đang cấu hình...
Kaggle API key configured.
Using device: cuda


In [ ]:
# @title 2. Import dataset MedQuAD from Kaggle

# Define the Kaggle dataset path
dataset_id = "gpreda/medquad"

# Dataset Directory
output_dir = "medquad_data"
os.makedirs(output_dir, exist_ok=True)

print(f"Đang tải dataset: {dataset_id}...")
# Tải dataset từ Kaggle vào thư mục đã tạo
# Sử dụng ! để chạy lệnh shell, hoặc os.system
!kaggle datasets download -d {dataset_id} -p {output_dir} --unzip

print(f"Dataset {dataset_id} đã được tải và giải nén vào thư mục: {output_dir}")

# Liệt kê các file trong thư mục để xác nhận
print("\nCác file trong thư mục dataset:")
!ls {output_dir}

# Đường dẫn đến file CSV (dựa trên output của lệnh ls)
# Giả sử tên file là medquad.csv như trong code của bạn
csv_file_path = os.path.join(output_dir, 'medquad.csv')

if not os.path.exists(csv_file_path):
    print(f"Lỗi: Không tìm thấy file {csv_file_path}. Vui lòng kiểm tra lại tên file trong thư mục {output_dir}.")
    # Liệt kê lại để người dùng kiểm tra
    !ls {output_dir}
    # Dừng ở đây nếu không tìm thấy file CSV
    # raise FileNotFoundError(f"{csv_file_path} not found.")
else:
    print(f"Tìm thấy file dữ liệu: {csv_file_path}")

Đang tải dataset: gpreda/medquad...
Dataset URL: https://www.kaggle.com/datasets/gpreda/medquad
License(s): apache-2.0
  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 341MB/s]
Dataset gpreda/medquad đã được tải và giải nén vào thư mục: medquad_data

Các file trong thư mục dataset:
medquad.csv
Tìm thấy file dữ liệu: medquad_data/medquad.csv


In [ ]:
# @title 3. Import necessary dependencies and settings GPU
from transformers import (
    GPT2TokenizerFast,
    GPT2ForQuestionAnswering,
    Trainer,
    TrainingArguments,
    default_data_collator
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# @title 4 Preprocessing MedQuAD

import pandas as pd
from datasets import Dataset, DatasetDict # Đảm bảo import này nếu cell chạy độc lập
import random
import os
import re # Thư viện regex
import html # Import html để unescape HTML entities
from bs4 import BeautifulSoup # Import BeautifulSoup để strip HTML tags
# import matplotlib.pyplot as plt # Bỏ comment và cài đặt nếu muốn vẽ đồ thị
# import seaborn as sns # Bỏ comment và cài đặt nếu muốn vẽ đồ thị

# --- KHAI BÁO BIẾN VÀ HÀM HỖ TRỢ ---
processed_data_generative = []
question_col = 'question'
answer_col = 'answer'
# csv_file_path được giả định là đã được định nghĩa ở Cell 2

# Hàm chuẩn hóa khoảng trắng và dòng mới
def normalize_whitespace(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.replace('\\n', ' ') # Xử lý ký tự '\n' trong chuỗi
    text = re.sub(r'\s+', ' ', text) # Thay thế nhiều khoảng trắng/tab/dòng mới bằng một khoảng trắng
    return text.strip() # Loại bỏ khoảng trắng ở đầu và cuối

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)

    # Bước 1: Loại bỏ HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_no_html = soup.get_text()

    # Bước 2: Unescape HTML entities
    text_unescaped = html.unescape(text_no_html)

    # Bước 3: Chuẩn hóa khoảng trắng và dòng mới
    text_normalized_space = normalize_whitespace(text_unescaped)

    # Bước 4: Chuyển về chữ thường (lowercase)
    text_lowercased = text_normalized_space.lower()

    return text_lowercased

# --- BẮT ĐẦU XỬ LÝ DỮ LIỆU ---

# Kiểm tra xem csv_file_path có tồn tại và không None (được định nghĩa từ Cell 2)
if 'csv_file_path' in globals() and csv_file_path and os.path.exists(csv_file_path):
    try:
        print(f"Đang đọc file dữ liệu từ: {csv_file_path}")
        df_original = pd.read_csv(csv_file_path)

        print("\n--- MÔ TẢ DỮ LIỆU GỐC (TRƯỚC KHI XỬ LÝ) ---")
        print(f"Số dòng và cột ban đầu: {df_original.shape}")
        # df_original.info() # Có thể hơi dài, cân nhắc nếu muốn hiển thị
        print("\nNăm dòng đầu tiên của dữ liệu gốc:")
        print(df_original.head())
        print(f"\nSố lượng câu hỏi rỗng ban đầu (NaN): {df_original[question_col].isnull().sum()}")
        print(f"Số lượng câu trả lời rỗng ban đầu (NaN): {df_original[answer_col].isnull().sum()}")

        # (Tùy chọn) Phân tích độ dài ban đầu (bỏ comment nếu cần)
        # df_original['original_question_len'] = df_original[question_col].astype(str).apply(len)
        # df_original['original_answer_len'] = df_original[answer_col].astype(str).apply(len)
        # print("\nThống kê độ dài câu hỏi/trả lời gốc:")
        # print(df_original[['original_question_len', 'original_answer_len']].describe())
        # plt.figure(figsize=(12, 5))
        # plt.subplot(1, 2, 1); sns.histplot(df_original['original_question_len'], bins=50, kde=False); plt.title('Phân bố Độ dài Câu hỏi Gốc')
        # plt.subplot(1, 2, 2); sns.histplot(df_original['original_answer_len'], bins=50, kde=False); plt.title('Phân bố Độ dài Câu trả lời Gốc')
        # plt.tight_layout(); plt.show()

        # --- BẮT ĐẦU CÁC BƯỚC TIỀN XỬ LÝ ---
        df_processed = df_original.copy()
        initial_rows_before_any_processing = len(df_processed)

        # 1. Loại bỏ các dòng mà question hoặc answer hoàn toàn là NaN (nếu có)
        df_processed.dropna(subset=[question_col, answer_col], inplace=True)
        rows_after_dropna = len(df_processed)
        print(f"\nĐã loại bỏ {initial_rows_before_any_processing - rows_after_dropna} dòng có giá trị NaN ở question hoặc answer.")

        # 2. Áp dụng tiền xử lý văn bản (bao gồm strip HTML, unescape, normalize whitespace, lowercase)
        print("\nĐang chuẩn hóa văn bản cho cột 'question' và 'answer'...")
        df_processed.loc[:, question_col] = df_processed[question_col].apply(preprocess_text)
        df_processed.loc[:, answer_col] = df_processed[answer_col].apply(preprocess_text)
        print("Hoàn tất chuẩn hóa văn bản.")

        # 3. Loại bỏ các dòng mà question hoặc answer trở thành rỗng SAU KHI chuẩn hóa
        # (ví dụ: nếu ban đầu chỉ chứa HTML hoặc khoảng trắng)
        df_processed = df_processed[df_processed[question_col].str.strip().astype(bool)]
        df_processed = df_processed[df_processed[answer_col].str.strip().astype(bool)]
        rows_after_empty_filter = len(df_processed)
        print(f"Đã loại bỏ {rows_after_dropna - rows_after_empty_filter} dòng có question/answer rỗng sau khi chuẩn hóa.")


        # 4. Loại bỏ trùng lặp DỰA TRÊN CỘT ĐÃ CHUẨN HÓA
        rows_before_final_dedup = len(df_processed)
        df_processed.drop_duplicates(subset=[question_col, answer_col], keep='first', inplace=True)
        rows_after_final_dedup = len(df_processed)
        print(f"Đã loại bỏ {rows_before_final_dedup - rows_after_final_dedup} dòng trùng lặp sau khi chuẩn hóa văn bản.")

        print("\n--- MÔ TẢ DỮ LIỆU SAU KHI TIỀN XỬ LÝ HOÀN CHỈNH ---")
        print(f"Số dòng và cột cuối cùng sau tất cả các bước xử lý: {df_processed.shape}")
        print("\nNăm dòng đầu tiên của dữ liệu đã xử lý hoàn chỉnh:")
        print(df_processed.head())

        # (Tùy chọn) Phân tích độ dài sau xử lý (bỏ comment nếu cần)
        # df_processed['processed_question_len'] = df_processed[question_col].astype(str).apply(len)
        # df_processed['processed_answer_len'] = df_processed[answer_col].astype(str).apply(len)
        # print("\nThống kê độ dài câu hỏi/trả lời sau xử lý:")
        # print(df_processed[['processed_question_len', 'processed_answer_len']].describe())
        # plt.figure(figsize=(12, 5))
        # plt.subplot(1, 2, 1); sns.histplot(df_processed['processed_question_len'], bins=50, kde=False); plt.title('Phân bố Độ dài Câu hỏi Đã Xử Lý')
        # plt.subplot(1, 2, 2); sns.histplot(df_processed['processed_answer_len'], bins=50, kde=False); plt.title('Phân bố Độ dài Câu trả lời Đã Xử Lý')
        # plt.tight_layout(); plt.show()

        # --- CHUẨN BỊ DỮ LIỆU CHO HUẤN LUYỆN ---
        # Chuyển df_processed thành danh sách dictionaries
        for index, row in df_processed.iterrows():
            # (Tùy chọn) Bạn có thể thêm lại bộ lọc độ dài ở đây nếu muốn,
            # dựa trên độ dài của question_text và answer_text đã được chuẩn hóa.
            # MIN_Q_LEN = 5; MAX_Q_LEN = 250; MIN_A_LEN = 10; MAX_A_LEN = 700
            # question_text = row[question_col]
            # answer_text = row[answer_col]
            # if not (MIN_Q_LEN < len(question_text) < MAX_Q_LEN and \
            #         MIN_A_LEN < len(answer_text) < MAX_A_LEN):
            #     continue # Bỏ qua nếu không đạt yêu cầu độ dài

            processed_data_generative.append({
                "id": str(df_original.index[index]) if index in df_original.index else str(index), # Cố gắng giữ ID gốc nếu có thể
                "question": row[question_col],
                "answer": row[answer_col]
            })

        print(f"\nSố lượng cặp Q&A hợp lệ để đưa vào huấn luyện: {len(processed_data_generative)}")

        if not processed_data_generative:
            # Điều này không nên xảy ra nếu df_processed có dữ liệu
            raise ValueError("Không có dữ liệu nào được xử lý từ CSV để huấn luyện.")

    except Exception as e:
        print(f"Lỗi khi đọc hoặc xử lý file CSV: {e}")
        import traceback
        traceback.print_exc()
        # Fallback nếu có lỗi
        processed_data_generative = [
            {"id": "syn_0", "question": "what are the symptoms of flu?", "answer": "symptoms of flu include fever, cough, sore throat, runny or stuffy nose, body aches, headache, chills, and fatigue."},
            {"id": "syn_1", "question": "how to treat a common cold?", "answer": "to treat a common cold, get plenty of rest, drink fluids, and use over-the-counter medications for symptoms."},
        ]
        print("Sử dụng dữ liệu giả lập do lỗi xử lý.")
else:
    if 'csv_file_path' not in globals() or not csv_file_path:
        print(f"Đường dẫn file CSV ('csv_file_path') không được cung cấp (có thể do lỗi ở Cell 2). Sử dụng dữ liệu giả lập.")
    else: # csv_file_path có giá trị nhưng file không tồn tại
        print(f"File {csv_file_path} không tồn tại. Sử dụng dữ liệu giả lập.")
    # Fallback nếu không có file CSV
    processed_data_generative = [
        {"id": "syn_0", "question": "what are the symptoms of flu?", "answer": "symptoms of flu include fever, cough, sore throat, runny or stuffy nose, body aches, headache, chills, and fatigue."},
        {"id": "syn_1", "question": "how to treat a common cold?", "answer": "to treat a common cold, get plenty of rest, drink fluids, and use over-the-counter medications for symptoms."},
    ]

# --- GIỚI HẠN SỐ LƯỢNG MẪU VÀ TẠO DATASETDICT ---
# (Giữ nguyên phần này từ code gốc của bạn)
MAX_SAMPLES_TO_USE = 17000
if len(processed_data_generative) > MAX_SAMPLES_TO_USE:
    print(f"\nGiới hạn dữ liệu huấn luyện xuống còn {MAX_SAMPLES_TO_USE} mẫu để demo.")
    processed_data_generative = random.sample(processed_data_generative, MAX_SAMPLES_TO_USE)
elif not processed_data_generative: # Kiểm tra lại nếu processed_data_generative rỗng (ví dụ, do lỗi fallback)
     print("CẢNH BÁO: processed_data_generative rỗng sau khi lấy mẫu hoặc do lỗi fallback. Kiểm tra lại.")
     # Tạo lại dữ liệu giả lập nếu rỗng hoàn toàn
     processed_data_generative = [
        {"id": "syn_fallback_0", "question": "example question for empty case?", "answer": "example answer for empty case."},
    ]

if not processed_data_generative: # Kiểm tra cuối cùng
    raise ValueError("Không có dữ liệu để xử lý (kể cả dữ liệu giả lập).")

print(f"\nSố lượng mẫu cuối cùng được sử dụng: {len(processed_data_generative)}")

dataset_gen = Dataset.from_pandas(pd.DataFrame(processed_data_generative))

# Chia train/validation
train_test_split_gen = dataset_gen.train_test_split(test_size=0.1, seed=42) # seed=42 để đảm bảo chia giống nhau mỗi lần chạy
dataset_dict_gen = DatasetDict({
    'train': train_test_split_gen['train'],
    'validation': train_test_split_gen['test']
})

print("\nCấu trúc dữ liệu cho Generative QA (sau khi chuẩn hóa và lấy mẫu):")
print(dataset_dict_gen)
print("\nVí dụ một mẫu dữ liệu (từ tập train đã chuẩn hóa):")
if len(dataset_dict_gen["train"]) > 0:
    sample_example_gen = dataset_dict_gen["train"][0]
    print(f"ID: {sample_example_gen['id']}")
    print(f"Question: {sample_example_gen['question']}")
    print(f"Answer: {sample_example_gen['answer']}")
else:
    print("Tập huấn luyện rỗng!")

Đang đọc file dữ liệu từ: medquad_data/medquad.csv

--- MÔ TẢ DỮ LIỆU GỐC (TRƯỚC KHI XỬ LÝ) ---
Số dòng và cột ban đầu: (16412, 4)

Năm dòng đầu tiên của dữ liệu gốc:
                                 question  \
0                What is (are) Glaucoma ?   
1                  What causes Glaucoma ?   
2     What are the symptoms of Glaucoma ?   
3  What are the treatments for Glaucoma ?   
4                What is (are) Glaucoma ?   

                                              answer           source  \
0  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   
1  Nearly 2.7 million people have glaucoma, a lea...  NIHSeniorHealth   
2  Symptoms of Glaucoma  Glaucoma can develop in ...  NIHSeniorHealth   
3  Although open-angle glaucoma cannot be cured, ...  NIHSeniorHealth   
4  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   

  focus_area  
0   Glaucoma  
1   Glaucoma  
2   Glaucoma  
3   Glaucoma  
4   Glaucoma  

Số lượng câu hỏi rỗng ban đầu (N

In [ ]:
# @title 5. Import Tokenizer and GPT-2
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, DataCollatorForLanguageModeling

model_name = "gpt2" # Có thể thử "gpt2-medium" nếu có tài nguyên
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

# GPT-2 tokenizer thường không có pad_token, đặt nó thành eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Sử dụng GPT2LMHeadModel cho việc sinh văn bản
model = GPT2LMHeadModel.from_pretrained(model_name)

# Quan trọng: resize token embeddings nếu bạn đã thêm special tokens vào tokenizer
# (Trong trường hợp này, chúng ta không thêm token mới mà dùng eos_token có sẵn)
# model.resize_token_embeddings(len(tokenizer))


# Chuyển model lên GPU nếu có
model.to(device)
print("Tokenizer và GPT2LMHeadModel đã được tải.")
print(f"EOS token: '{tokenizer.eos_token}', ID: {tokenizer.eos_token_id}")
print(f"PAD token: '{tokenizer.pad_token}', ID: {tokenizer.pad_token_id}")

# Định nghĩa separator token dựa trên tokenizer đã tải
separator_token = tokenizer.eos_token # Hoặc một token khác nếu bạn muốn, ví dụ " <|sep|> " và add nó vào tokenizer
                                   # Nhưng dùng eos_token là cách đơn giản.
                                   # Tuy nhiên, eos_token cũng dùng để báo hiệu kết thúc chuỗi.
                                   # Một lựa chọn khác là dùng một format cụ thể hơn:
                                   # "Question: [Q] Answer: [A]"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Tokenizer và GPT2LMHeadModel đã được tải.
EOS token: '<|endoftext|>', ID: 50256
PAD token: '<|endoftext|>', ID: 50256


In [ ]:
# @title 6. Tokenization for trainer (Generative QA - Padding đến max_length)

# Đặt một giới hạn trên thực tế, ví dụ: giới hạn của mô hình
# hoặc một giá trị bạn biết GPU có thể xử lý
effective_max_length = tokenizer.model_max_length # Thường là 1024 cho GPT-2 base
# Hoặc bạn có thể đặt một giá trị nhỏ hơn nếu muốn, ví dụ: 512
# effective_max_length = 512

print(f"Sử dụng effective_max_length (cho truncation và padding): {effective_max_length}")

def preprocess_function_generative(examples):
    inputs_text = []
    for q, a in zip(examples["question"], examples["answer"]):
        text = f"Question: {q} Answer: {a}{tokenizer.eos_token}"
        inputs_text.append(text)

    model_inputs = tokenizer(
        inputs_text,
        truncation=True,
        padding="max_length",   # QUAN TRỌNG: Pad đến max_length
        max_length=effective_max_length,
        return_attention_mask=True # Mặc định là True, nhưng có thể thêm tường minh
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs


# Áp dụng hàm tiền xử lý cho dataset
column_names_gen_train = dataset_dict_gen["train"].column_names
tokenized_datasets_gen = dataset_dict_gen.map(
    preprocess_function_generative,
    batched=True,
    remove_columns=column_names_gen_train
)

print("\nCấu trúc dữ liệu đã token hóa cho Generative QA (Padded to max_length):")
sample_tokenized_gen = tokenized_datasets_gen["train"][0]
print(f"Keys: {sample_tokenized_gen.keys()}")
print(f"Length of Input IDs: {len(sample_tokenized_gen['input_ids'])}")
print(f"Length of Labels: {len(sample_tokenized_gen['labels'])}")
print(f"Length of Attention Mask: {len(sample_tokenized_gen['attention_mask'])}")

# Kiểm tra một vài mẫu khác nhau để đảm bảo độ dài bằng nhau
if len(tokenized_datasets_gen["train"]) > 1:
    sample2_tokenized_gen = tokenized_datasets_gen["train"][1]
    print(f"Length of Input IDs (sample 2): {len(sample2_tokenized_gen['input_ids'])}")


# ... (phần còn lại của cell 6) ...

Sử dụng effective_max_length (cho truncation và padding): 1024


Map:   0%|          | 0/14723 [00:00<?, ? examples/s]

Map:   0%|          | 0/1636 [00:00<?, ? examples/s]


Cấu trúc dữ liệu đã token hóa cho Generative QA (Padded to max_length):
Keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
Length of Input IDs: 1024
Length of Labels: 1024
Length of Attention Mask: 1024
Length of Input IDs (sample 2): 1024


In [ ]:
# @title 7. Training (Fine-tuning) GPT 2 (Generative QA)

import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Trainer, TrainingArguments

# Data collator sẽ xử lý padding và tạo labels phù hợp cho language modeling
# Nếu labels chưa được set -100 cho padding tokens, DataCollatorForLanguageModeling sẽ làm điều đó
data_collator_gen = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Chúng ta đang làm Causal LM (CLM), không phải Masked LM (MLM)
)

training_args_gen = TrainingArguments(
    output_dir="./results_medquad_gpt2_generative",
    eval_strategy="epoch",
    learning_rate=5e-5, # Có thể cần điều chỉnh
    per_device_train_batch_size=6, # Giảm nếu OOM
    per_device_eval_batch_size=6,  # Giảm nếu OOM
    gradient_accumulation_steps=4,
    num_train_epochs=3, # Tăng lên 3-5 epochs hoặc hơn cho kết quả tốt hơn, nhưng tốn thời gian
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none",
    # Thêm các tham số khác nếu cần, ví dụ: gradient_accumulation_steps
)

trainer_gen = Trainer(
    model=model, # model là GPT2LMHeadModel
    args=training_args_gen,
    train_dataset=tokenized_datasets_gen["train"],
    eval_dataset=tokenized_datasets_gen["validation"],
    data_collator=data_collator_gen,
)

print("\nBắt đầu huấn luyện mô hình Generative QA...")
try:
    trainer_gen.train()
    print("Huấn luyện hoàn tất.")
except Exception as e:
    print(f"Lỗi trong quá trình huấn luyện: {e}")
    print("Thử giảm batch_size nếu là lỗi OOM.")

# Lưu model và tokenizer tốt nhất
output_model_dir_gen = "./fine_tuned_medquad_gpt2_generative_final"
trainer_gen.save_model(output_model_dir_gen)
tokenizer.save_pretrained(output_model_dir_gen) # Lưu cả tokenizer
print(f"Model và tokenizer (Generative) đã được lưu vào thư mục: {output_model_dir_gen}")


Bắt đầu huấn luyện mô hình Generative QA...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.718000,1.645040
2,1.614900,1.564053
3,1.627300,1.544399


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Huấn luyện hoàn tất.
Model và tokenizer (Generative) đã được lưu vào thư mục: ./fine_tuned_medquad_gpt2_generative_final


In [ ]:
# @title 8. Model Evaluation (ROUGE)
import evaluate
import numpy as np
from tqdm import tqdm # Để theo dõi tiến trình

# Tải metric ROUGE
rouge_metric = evaluate.load("rouge")

# Sử dụng mô hình đã được fine-tune tốt nhất
eval_model = trainer_gen.model
eval_tokenizer = tokenizer # Hoặc tải từ thư mục đã lưu

# Lấy tập validation (hoặc một tập test riêng nếu có)
validation_dataset = dataset_dict_gen["validation"] # Sử dụng lại tập validation đã chia ở Cell 4
# Nếu bạn muốn, bạn có thể tạo một phần nhỏ hơn để đánh giá nhanh hơn
# small_validation_dataset = validation_dataset.select(range(100)) # Ví dụ: 100 mẫu

predictions = []
references = []

print(f"Bắt đầu sinh câu trả lời trên tập validation ({len(validation_dataset)} mẫu) để đánh giá...")
# Đặt model ở chế độ eval
eval_model.eval()
eval_model.to(device) # Đảm bảo model trên đúng device

for example in tqdm(validation_dataset):
    question = example["question"]
    reference_answer = example["answer"]

    # Tạo prompt giống như khi huấn luyện/suy luận
    prompt = f"Question: {question} Answer:"
    inputs = eval_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device) # Giới hạn max_length cho prompt

    # Sinh câu trả lời
    try:
        with torch.no_grad():
            # Tăng max_length ở đây để cho phép câu trả lời dài hơn prompt ban đầu
            # Điều chỉnh các tham số generate cho phù hợp với cách bạn dùng trong Gradio
            output_sequences = eval_model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask, # Quan trọng khi input_ids có padding (mặc dù ở đây truncation=True cho prompt)
                max_length=len(inputs.input_ids[0]) + 150,  # Độ dài tối đa của (prompt + câu trả lời)
                num_beams=1, # Giảm num_beams để nhanh hơn khi đánh giá nhiều mẫu
                no_repeat_ngram_size=2,
                early_stopping=True,
                eos_token_id=eval_tokenizer.eos_token_id,
                pad_token_id=eval_tokenizer.pad_token_id
            )

        # Decode câu trả lời
        # Cần xử lý để chỉ lấy phần "Answer:"
        generated_text_full = eval_tokenizer.decode(output_sequences[0], skip_special_tokens=False)
        answer_prefix = "Answer:"
        start_of_answer_marker = generated_text_full.rfind(answer_prefix)

        if start_of_answer_marker != -1:
            start_of_answer_idx = start_of_answer_marker + len(answer_prefix)
            predicted_answer = generated_text_full[start_of_answer_idx:].strip()
            if predicted_answer.endswith(eval_tokenizer.eos_token):
                predicted_answer = predicted_answer[:-len(eval_tokenizer.eos_token)].strip()
        else:
            # Fallback: cố gắng loại bỏ prompt ban đầu
            if generated_text_full.startswith(prompt):
                 predicted_answer = generated_text_full[len(prompt):].strip()
                 if predicted_answer.endswith(eval_tokenizer.eos_token):
                    predicted_answer = predicted_answer[:-len(eval_tokenizer.eos_token)].strip()
            else:
                predicted_answer = eval_tokenizer.decode(output_sequences[0], skip_special_tokens=True) # Lấy hết nếu không chắc
                # Loại bỏ prompt một cách thô sơ hơn
                if predicted_answer.startswith(question):
                    predicted_answer = predicted_answer[len(question):].strip().lstrip(':').strip()

        predictions.append(predicted_answer if predicted_answer else " ") # Tránh chuỗi rỗng cho ROUGE
        references.append(reference_answer if reference_answer else " ")

    except Exception as e:
        print(f"Lỗi khi sinh câu trả lời cho câu hỏi: '{question}'. Lỗi: {e}")
        predictions.append(" ") # Thêm một dự đoán rỗng/mặc định nếu có lỗi
        references.append(reference_answer if reference_answer else " ")


if predictions and references:
    print("\nĐang tính toán ROUGE scores...")
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)
    print("\nKết quả đánh giá ROUGE:")
    for key, value in rouge_results.items():
        print(f"{key}: {value*100:.2f}") # Nhân 100 để ra % nếu muốn
else:
    print("Không có dự đoán hoặc tham chiếu nào để tính ROUGE.")



In [ ]:
# @title 9. Gradio interface (Generative QA use Sentence Transformers)

import gradio as gr
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util # Thư viện chính cho tìm kiếm ngữ nghĩa

# --- PHẦN KHỞI TẠO CHO SENTENCE TRANSFORMERS (CHẠY MỘT LẦN) ---
# Chọn mô hình Sentence Transformer
# model_st_name = 'all-MiniLM-L6-v2' # Nhanh, chất lượng khá, kích thước nhỏ
model_st_name = 'all-mpnet-base-v2'  # Chất lượng cao hơn, kích thước lớn hơn một chút
print(f"Đang tải mô hình Sentence Transformer: {model_st_name}...")
try:
    st_model = SentenceTransformer(model_st_name)
    print("Mô hình Sentence Transformer đã tải xong.")
except Exception as e:
    print(f"Lỗi khi tải mô hình Sentence Transformer: {e}. Vui lòng đảm bảo thư viện đã được cài đặt và có kết nối mạng.")
    st_model = None # Đặt là None nếu không tải được

# Chuẩn bị dữ liệu (từ knowledge_base_df_gen của bạn từ Cell 4)
# Đảm bảo processed_data_generative đã được định nghĩa và chứa dữ liệu
if 'processed_data_generative' not in globals() or not processed_data_generative:
    print("CẢNH BÁO: processed_data_generative không tìm thấy hoặc rỗng. Chức năng tìm kiếm tương đồng sẽ bị hạn chế.")
    knowledge_base_df_gen = pd.DataFrame([
        {"id": "syn_kb_0", "question": "What are the symptoms of flu?", "answer": "Flu is a common respiratory illness caused by influenza viruses."},
        {"id": "syn_kb_1", "question": "How can one treat a common cold?", "answer": "For a common cold, it's advisable to get plenty of rest, drink fluids, and use over-the-counter medications for symptoms."},
        {"id": "syn_kb_2", "question": "Tell me about MRI scans.", "answer": "MRI, or Magnetic Resonance Imaging, is a medical imaging technique used to form pictures of the anatomy and physiological processes of the body."}
    ])
else:
    knowledge_base_df_gen = pd.DataFrame(processed_data_generative)

corpus_questions = []
corpus_answers = []
corpus_embeddings = None

if st_model and not knowledge_base_df_gen.empty:
    corpus_questions = knowledge_base_df_gen['question'].astype(str).tolist()
    corpus_answers = knowledge_base_df_gen['answer'].astype(str).tolist()
    if corpus_questions:
        print(f"Đang tính toán embeddings cho {len(corpus_questions)} câu hỏi trong corpus...")
        try:
            # Chuyển st_model lên GPU nếu có và torch.cuda.is_available()
            if torch.cuda.is_available():
                st_model.to(device) # device đã được định nghĩa ở các cell trước
                print(f"Sentence Transformer model moved to {device}.")

            corpus_embeddings = st_model.encode(corpus_questions, convert_to_tensor=True, show_progress_bar=True)
            print("Đã tính toán xong embeddings cho corpus.")
        except Exception as e:
            print(f"Lỗi khi tính toán embeddings cho corpus: {e}")
            corpus_embeddings = None # Đặt là None nếu có lỗi
    else:
        print("Không có câu hỏi trong corpus để tính embeddings.")
else:
    if not st_model:
        print("Không thể tính embeddings do mô hình Sentence Transformer chưa được tải.")
    if knowledge_base_df_gen.empty:
        print("Knowledge base rỗng, không có dữ liệu để tính embeddings.")
# --- KẾT THÚC PHẦN KHỞI TẠO CHO SENTENCE TRANSFORMERS ---


# Sử dụng model generative QA từ trainer nếu huấn luyện vừa xong
# Đảm bảo trainer_gen đã được huấn luyện và model của nó có sẵn
if 'trainer_gen' not in globals() or not hasattr(trainer_gen, 'model'):
    print("CẢNH BÁO: trainer_gen hoặc trainer_gen.model không tìm thấy. Cần tải lại model generative.")
    # Tạm thời tải lại model generative đã lưu nếu trainer không có sẵn
    # (Điều này chỉ là fallback, lý tưởng là dùng model từ trainer)
    try:
        output_model_dir_gen = "./fine_tuned_medquad_gpt2_generative_final"
        gen_qa_model = GPT2LMHeadModel.from_pretrained(output_model_dir_gen)
        gen_qa_tokenizer = GPT2TokenizerFast.from_pretrained(output_model_dir_gen)
        gen_qa_model.to(device)
        print(f"Đã tải lại model generative QA từ {output_model_dir_gen}")
    except Exception as e:
        print(f"Lỗi khi tải lại model generative QA: {e}. Chức năng trả lời sẽ không hoạt động.")
        gen_qa_model = None
        gen_qa_tokenizer = None
else:
    gen_qa_model = trainer_gen.model
    gen_qa_tokenizer = tokenizer # tokenizer đã được định nghĩa và lưu

if gen_qa_model:
    gen_qa_model.eval() # Chuyển model sang chế độ evaluation


def find_most_similar_qa_st(user_question, top_k=1):
    """
    Tìm cặp Q&A tương tự nhất sử dụng Sentence Transformers và cosine similarity.
    """
    if not st_model or corpus_embeddings is None or not corpus_questions:
        print("Tìm kiếm tương đồng không khả dụng do thiếu st_model, corpus_embeddings hoặc corpus_questions.")
        return None, None, 0

    try:
        # Tính embedding cho câu hỏi của người dùng
        question_embedding = st_model.encode(user_question, convert_to_tensor=True)
        if torch.cuda.is_available(): # Đảm bảo embedding của câu hỏi cũng trên GPU nếu corpus_embeddings trên GPU
             question_embedding = question_embedding.to(corpus_embeddings.device)


        # Tính cosine similarity
        # util.cos_sim yêu cầu cả hai tensor trên cùng một device
        cosine_scores = util.cos_sim(question_embedding, corpus_embeddings)[0]

        # Tìm top_k kết quả
        # Chuyển cosine_scores về CPU trước khi dùng topk nếu nó đang trên GPU
        # và bạn muốn xử lý chỉ số trên CPU, hoặc giữ trên GPU nếu các bước sau cũng trên GPU.
        # Đối với lấy chỉ số và truy cập list Python, CPU là phù hợp.
        top_results = torch.topk(cosine_scores.cpu(), k=min(top_k, len(corpus_questions)))

        if not top_results.indices.nelement(): # Kiểm tra nếu không có kết quả nào
            return None, None, 0

        best_idx = top_results.indices[0].item()
        best_score = top_results.values[0].item()

        # Ngưỡng tương đồng (có thể điều chỉnh)
        # Với cosine similarity, ngưỡng thường cao hơn (0.0 đến 1.0)
        similarity_threshold = 0.65
        if best_score > similarity_threshold:
            best_match_question = corpus_questions[best_idx]
            best_match_answer = corpus_answers[best_idx] # Lấy câu trả lời tương ứng
            return best_match_question, best_match_answer, best_score
        return None, None, 0 # Trả về None nếu không có gì trên ngưỡng
    except Exception as e:
        print(f"Lỗi trong find_most_similar_qa_st: {e}")
        return None, None, 0


def answer_question_gradio_generative(user_question):
    print(f"\nNhận câu hỏi từ Gradio (Generative): {user_question}")
    if not user_question.strip():
        return "Vui lòng nhập câu hỏi."

    # Bước 1: Tìm Q&A tương tự trong "cơ sở dữ liệu" bằng Sentence Transformers
    similar_q, similar_a, similarity_score = find_most_similar_qa_st(user_question)
    similarity_info = ""
    if similar_q:
        similarity_info = (
            f"\n\n--- Thông tin tham khảo từ dữ liệu (Tương đồng ngữ nghĩa: {similarity_score:.2f}) ---\n"
            f"Câu hỏi tương tự nhất:\n{similar_q}\n"
            f"Câu trả lời tương ứng:\n{similar_a}\n"
            f"-------------------------------------------------------------------"
        )
        print(f"Tìm thấy Q&A tương tự (ngữ nghĩa) với độ tương đồng {similarity_score:.2f}")


    # Bước 2: Sinh câu trả lời bằng mô hình generative QA
    if not gen_qa_model or not gen_qa_tokenizer:
        return "Lỗi: Mô hình generative QA chưa được tải."

    prompt = f"Question: {user_question} Answer:"
    inputs = gen_qa_tokenizer(prompt, return_tensors="pt").to(device if gen_qa_model.device.type == 'cuda' else 'cpu') # Đảm bảo input cùng device với model

    generated_answer_text = "Không thể sinh câu trả lời."
    try:
        with torch.no_grad():
            outputs = gen_qa_model.generate(
                inputs.input_ids,
                max_length=200,  # Tăng một chút để có không gian cho câu trả lời dài hơn
                num_beams=1,
                no_repeat_ngram_size=2, # Giúp tránh lặp lại cụm từ
                early_stopping=True,
                eos_token_id=gen_qa_tokenizer.eos_token_id,
                pad_token_id=gen_qa_tokenizer.pad_token_id,
                temperature=0.3, # Kiểm soát sự ngẫu nhiên
                do_sample=True,       # BẮT BUỘC PHẢI CÓ
                # top_k=50              # Nên thêm top_k hoặc top_p để tránh sinh ra token quá lạ
                # top_k=50,
                top_p=0.95
            )

        generated_text_full = gen_qa_tokenizer.decode(outputs[0], skip_special_tokens=False)
        answer_prefix = "Answer:"
        # Tìm vị trí của "Answer:" cuối cùng để đảm bảo lấy đúng phần trả lời nếu prompt có từ "Answer:"
        start_of_answer_marker = generated_text_full.rfind(answer_prefix)

        if start_of_answer_marker != -1:
            start_of_answer_idx = start_of_answer_marker + len(answer_prefix)
            generated_answer_text = generated_text_full[start_of_answer_idx:].strip()
            # Loại bỏ eos_token nếu có ở cuối
            if generated_answer_text.endswith(gen_qa_tokenizer.eos_token):
                generated_answer_text = generated_answer_text[:-len(gen_qa_tokenizer.eos_token)].strip()
        else: # Fallback nếu không tìm thấy "Answer:"
            if generated_text_full.startswith(prompt): # Nếu output bắt đầu bằng prompt
                 generated_answer_text = generated_text_full[len(prompt):].strip()
                 if generated_answer_text.endswith(gen_qa_tokenizer.eos_token):
                    generated_answer_text = generated_answer_text[:-len(gen_qa_tokenizer.eos_token)].strip()
            else: # Nếu không, lấy toàn bộ output sau khi bỏ các token đặc biệt
                generated_answer_text = gen_qa_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                # Đôi khi prompt vẫn có thể còn sót, thử loại bỏ nếu user_question có trong đó
                if generated_answer_text.startswith(user_question):
                     generated_answer_text = generated_answer_text[len(user_question):].strip().lstrip(':').strip()


        print(f"Câu trả lời được sinh ra: {generated_answer_text}")

        final_response = f"Chatbot Y Tế (GPT-2 Generative) Trả Lời:\n{generated_answer_text}"
        final_response += similarity_info

        return final_response

    except Exception as e:
        print(f"Lỗi khi sinh câu trả lời: {e}")
        import traceback
        traceback.print_exc()
        error_response = f"Đã xảy ra lỗi khi sinh câu trả lời. {e}"
        error_response += similarity_info
        return error_response

# Tạo giao diện Gradio
iface_gen = gr.Interface(
    fn=answer_question_gradio_generative,
    inputs=gr.Textbox(lines=3, placeholder="Nhập câu hỏi y tế của bạn ở đây... Ví dụ: 'What are common treatments for high blood pressure?'"),
    outputs=gr.Text(label="Câu Trả Lời và Thông Tin Tham Khảo"),
    title="Chatbot Hỏi Đáp Y Tế (GPT-2 Generative + Tìm Kiếm Ngữ Nghĩa)",
    description="Nhập câu hỏi y tế. Chatbot sẽ sinh câu trả lời và hiển thị cặp Hỏi-Đáp tương tự nhất (về mặt ngữ nghĩa) từ dữ liệu đã học.",
    examples=[
        ["What are the symptoms of flu?"],
        ["How to treat a common cold?"],
        ["What is an MRI?"],
        ["Tell me about diabetes type 2."],
        ["What are common treatments for high blood pressure?"]
    ],
    allow_flagging='never' # Tắt flagging nếu không cần
)

print("\nĐang khởi chạy giao diện Gradio cho Generative QA (có tìm kiếm ngữ nghĩa)...")
iface_gen.launch(debug=True, share=True) # Chạy với debug=True nếu cần xem log chi tiết
# iface_gen.launch(share=True)

KeyboardInterrupt: 